In [ ]:
!gdown --id 1Y8EOFLIRCcKpe_e0pO03yCAosTRjRMtC

Downloading...
From: https://drive.google.com/uc?id=1Y8EOFLIRCcKpe_e0pO03yCAosTRjRMtC
To: /content/UTKFace.zip
347MB [00:03, 109MB/s]


In [ ]:
!unzip -q /content/UTKFace.zip -d data

In [ ]:
# To download checkpoints, Keras models, TFLite models
from google.colab import files
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import datetime

In [ ]:
n = len(os.listdir('/content/data/UTKFace'))
n

23708

In [ ]:
MODEL_INPUT_IMAGE_SIZE = [ 200 , 200 ]
TRAIN_TEST_SPLIT = 0.3

# This method will be mapped for each filename in `list_ds`. 
def parse_image( filename ):

    # Read the image from the filename and resize it.
    image_raw = tf.io.read_file( filename )
    image = tf.image.decode_jpeg( image_raw , channels=3 ) 
    image = tf.cast(image, tf.float32)
    image = tf.image.resize( image , MODEL_INPUT_IMAGE_SIZE )

    # Split the filename to get the age and the gender. Convert the age ( str ) and the gender ( str ) to dtype float32.
    parts = tf.strings.split( tf.strings.split( filename , '/' )[ 2 ] , '_' )

    # Normalize
    age = tf.strings.to_number( parts[ 0 ] ) / 116

    return image , age

# List all the image files in the given directory.
list_ds = tf.data.Dataset.list_files( 'data/UTKFace/*' , shuffle=True )

# Map `parse_image` method to all filenames.
dataset = list_ds.map( parse_image , num_parallel_calls=tf.data.AUTOTUNE )

In [ ]:
# Create train and test splits of the dataset.
num_examples_in_test_ds = int( dataset.cardinality().numpy() * TRAIN_TEST_SPLIT )

test_ds = dataset.take( num_examples_in_test_ds )
train_ds = dataset.skip( num_examples_in_test_ds )

print( 'Num examples in train ds {}'.format( train_ds.cardinality() ) )
print( 'Num examples in test ds {}'.format( test_ds.cardinality() ) )

Num examples in train ds 16596
Num examples in test ds 7112


In [ ]:
BATCH_SIZE = 128
IMG_SIZE = (200, 200)
IMG_SHAPE = IMG_SIZE + (3,)

train_ds = train_ds.batch( BATCH_SIZE )
test_ds = test_ds.batch( BATCH_SIZE )

In [ ]:
# Create the base model from the pre-trained model MobileNet V2
base_model = tf.keras.applications.mobilenet.MobileNet(input_shape=IMG_SHAPE,
                                              include_top=False,
                                              weights="imagenet"
                                              )

base_model.trainable = False

image_batch, label_batch = next(iter(train_ds))
feature_batch = base_model(image_batch)
print(feature_batch.shape)

base_model.summary()

17227776/17225924 [==============================] - 0s 0us/step
(128, 6, 6, 1024)
Model: "mobilenet_1.00_224"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 200, 200, 3)]     0         
_________________________________________________________________
conv1 (Conv2D)               (None, 100, 100, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 100, 100, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 100, 100, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 100, 100, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 100, 100, 32)      128       
_______________________________

In [ ]:
inputs = tf.keras.Input(shape=IMG_SHAPE)
x = tf.keras.applications.mobilenet.preprocess_input(inputs)
x = base_model(x, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = tf.keras.layers.Dense(1)(x)
model = tf.keras.Model(inputs, outputs)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 200, 200, 3)]     0         
_________________________________________________________________
tf.math.truediv (TFOpLambda) (None, 200, 200, 3)       0         
_________________________________________________________________
tf.math.subtract (TFOpLambda (None, 200, 200, 3)       0         
_________________________________________________________________
mobilenet_1.00_224 (Function (None, 6, 6, 1024)        3228864   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1024)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense (Dense)                (None, 1)                 1025  

In [ ]:
# Init ModelCheckpoint callback
save_dir_ = 'mobilenetv2'  
save_dir = save_dir_ + '/{epoch:02d}-{val_mae:.2f}.h5'
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint( 
    save_dir , 
    save_best_only=True , 
    monitor='val_mae' , 
    mode='min' , 
)

tb_log_name = 'mobilenetv2'
# Init TensorBoard Callback
logdir = os.path.join( "tb_logs" , tb_log_name )
tensorboard_callback = tf.keras.callbacks.TensorBoard( logdir )

# Init Early Stopping callback
early_stopping_callback = tf.keras.callbacks.EarlyStopping( monitor='val_mae' , patience=10 )

In [ ]:
# warm up

num_epochs = 1
learning_rate = 0.001

model.compile( 
    loss=tf.keras.losses.mean_absolute_error ,
    optimizer = tf.keras.optimizers.Adam( learning_rate ) , 
    metrics=[ 'mae' ]
)

model.fit( 
    train_ds, 
    epochs=num_epochs,  
    validation_data=test_ds 
    # callbacks=[ checkpoint_callback , tensorboard_callback , early_stopping_callback ]
)

130/130 [==============================] - 51s 351ms/step - loss: 0.5181 - mae: 0.5181 - val_loss: 0.2361 - val_mae: 0.2361


In [ ]:
# fine-tuning
num_epochs = 30
learning_rate = 0.0001
base_model.trainable = True # unfreeze the mobilenet backbone

model.compile( 
    loss=tf.keras.losses.mean_absolute_error ,
    optimizer = tf.keras.optimizers.Adam( learning_rate ) , 
    metrics=[ 'mae' ]
)

model.fit( 
    train_ds, 
    epochs=num_epochs,  
    validation_data=test_ds,
    callbacks=[ checkpoint_callback , tensorboard_callback , early_stopping_callback ]
)

Epoch 1/30
130/130 [==============================] - 140s 1s/step - loss: 0.1263 - mae: 0.1263 - val_loss: 0.0744 - val_mae: 0.0744
Epoch 2/30
130/130 [==============================] - 134s 1s/step - loss: 0.0720 - mae: 0.0720 - val_loss: 0.0606 - val_mae: 0.0606
Epoch 3/30
130/130 [==============================] - 134s 1s/step - loss: 0.0646 - mae: 0.0646 - val_loss: 0.0616 - val_mae: 0.0616
Epoch 4/30
130/130 [==============================] - 134s 1s/step - loss: 0.0626 - mae: 0.0626 - val_loss: 0.0542 - val_mae: 0.0542
Epoch 5/30
 62/130 [=============>................] - ETA: 1:01 - loss: 0.0590 - mae: 0.0590

KeyboardInterrupt: ignored